Loading all necesarry libraries

In [3]:
# libraries
import pandas as pd
import numpy as np
import random
import math  
import tensorflow as tf
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import concatenate
from keras.layers import Dropout
from sklearn.metrics import r2_score
from google.colab import drive


ModuleNotFoundError: No module named 'sklearn'

In [ ]:
drive.mount('/content/drive') # mount google drive

Creating functions to create models

In [ ]:

# functions
# creating a function to create a new model
# fit model on dataset
def fit_model(train_feature, train_target, hidden_layer_size, epochs, validation_split = 0.2):
  # define model
  model = keras.Sequential([
    keras.layers.Dense(units=hidden_layer_size,  # first dense layer
                        kernel_initializer='lecun_normal',
                        activation='selu'),
    keras.layers.Dense(units=hidden_layer_size,  # second dense layer
                        kernel_initializer='lecun_normal',
                        activation='selu'),
    keras.layers.Dropout(0.2), # dropout layer
    keras.layers.Dense(1),  # output layer
  ])
  # optimiser for the model with loss function
  model.compile(loss=tf.keras.losses.mae, 
              optimizer = tf.keras.optimizers.Adam(), 
              metrics=["mae"])
  # training the model
  model.fit(train_feature,
            train_target, 
            epochs = epochs,
            validation_split = validation_split)
  
  return model

# make an ensemble prediction for regression
def ensemble_predictions(models, validation_feature):
  # make predictions
  yhats = [model.predict(validation_feature) for model in models]
  yhats = np.array(yhats)
  # find mean across ensemble members
  result = np.mean(yhats, axis=0)
  return result

# evaluate ensemble model
def evaluate_members(models, validation_feature, validation_target):
  yhat = ensemble_predictions(models, validation_feature.to_numpy()) # make prediction
  MSE = np.square(np.subtract(yhat, validation_target.to_numpy())).mean() # calculating MSE
  r2 = r2_score(validation_target.to_numpy(), yhat) # finding pseudo R2
  return math.sqrt(MSE); r2

# load models from file
def load_all_models(start_layers, finish_layers, number_of_same_model):
  all_models = list()
  for i in range(start_layers, finish_layers + 1):
    for j in range(number_of_same_model):
      # define filename for this ensemble
      filename = '/content/drive/MyDrive/Colab_Notebooks/Models_460/Models/model_' + str(j + 1) + '_hidden_layers_' + str(i) + '.h5'
      # load model from file
      model = load_model(filename)
      model.trainable = False # freezing the model
      model.compile(loss=tf.keras.losses.mae, 
              optimizer = tf.keras.optimizers.Adam(), 
              metrics=["mae"]) # recompiling the model
      # add to list of members
      all_models.append(model) # adding the models to the list
  return all_models

# define stacked model from multiple member input models
def define_stacked_model(models):
    # define multi-headed input
    ensemble_visible = [model.input for model in models] # creating a list of inputs for each model, should be 45 for all models
    # concatenate merge output from each model
    ensemble_outputs = [model.output for model in models] # creating a list of outputs for each model should be 1 for all models
    merge = concatenate(ensemble_outputs) # concatenating all models
    hidden_1 = Dense(100, activation='relu')(merge) # creating a dense layer of 100 neurons
    Dropout_1 = Dropout(0.2)(hidden_1) # creating a dropout layer with 0.2 dropout rate
    output = Dense(1)(Dropout_1) # creating a dense layer of 1 neuron for the output of a regression model
    model = Model(inputs = ensemble_visible, outputs = output) #creating a model with the inputs and outputs
    # compile
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error']) # compiling the model
    return model
  
  # fit a stacked model
def fit_stacked_model(model, feature_set, target_set):
	# prepare input data
	X = [feature_set for _ in range(len(model.input))]
	# training model
	model.fit(X, target_set, epochs = 20, validation_split = 0.2)

# make a prediction with a stacked model
def predict_stacked_model(model, feature_set, target_set):
  # prepare input data
  X = [feature_set for _ in range(len(model.input))]
  # make prediction
  yhat = model.predict(X, verbose=0)
  mse = np.square(np.subtract(yhat, target_set.to_numpy())).mean() # calculating MSE
  r2 = r2_score(target_set.to_numpy(), yhat) # finding pseudo R2
  return math.sqrt(mse), r2 

Loading data set

In [ ]:

# creating training set and validation
df = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/Models_460/Cleaned_Data_Python.csv') # Read the data

# Convert the date to datetime64
df['Transaction_date'] = pd.to_datetime(df['Transaction_date'], format='%Y-%m-%d')

validation_df = df.loc[(df['Transaction_date'] >= '2022-08-24')] # validation set
validation_df.drop('Transaction_date', axis = 1, inplace=True) # validation set with no dates
rest_df = df.loc[(df['Transaction_date'] < '2022-08-24')] # traing set
rest_df.drop('Transaction_date', axis = 1, inplace=True) # training set with no dates


target_validation = validation_df['Full_Price_Sales_Quantity'] # target validation
feature_validation = validation_df.drop('Full_Price_Sales_Quantity', axis = 1)  # feature validation
target_training = rest_df['Full_Price_Sales_Quantity'] # target training
features_training = rest_df.drop('Full_Price_Sales_Quantity', axis = 1) # feature training

Creating trained models

In [ ]:
start_layers = 25 # node size to start with
finish_layers = 45 # node size to finish with 45
number_of_same_model = 5 # number of same model to run
epochs = 10 # number of epochs
for i in range(start_layers, finish_layers + 1):
  for j in range(number_of_same_model):
  # training the model
    model = fit_model(features_training, target_training, hidden_layer_size = i, epochs = epochs)
    # creating a filename for the model
    filename = '/content/drive/MyDrive/Colab_Notebooks/Models_460/Models/model_' + str(j + 1) + '_hidden_layers_' + str(i) + '.h5' 
    # saving trained model to be used later
    model.save(filename) 

Creating prediction for validation set

Load all models

In [ ]:
# load all models
start_layers = 25 # node size to start with
finish_layers = 45 # node size to finish with 45
number_of_same_model = 5 # number of same model to run
models = load_all_models(start_layers, finish_layers, number_of_same_model) # loading all models


Train a new model by combaning all the other models

In [ ]:
# define ensemble model
stacked_model = define_stacked_model(models)
# training the emsemble model
fit_stacked_model(stacked_model, features_training, target_training)
stacked_model.save('/content/drive/MyDrive/Colab_Notebooks/Models_460/Models/best_model.h5') # saving trained model to be used later


Predicting the validation set

In [ ]:
# loading ensemble model
best_file_name = '/content/drive/MyDrive/Colab_Notebooks/Models_460/Models/best_model.h5' # file name of the best model
best_model = load_model(best_file_name) # lodading the best model
best_model.compile(loss='mae', optimizer='adam', metrics=['mae']) # compiling the best model
# plot_model(stacked_model, show_shapes=True, to_file='model_graph.png') # plotting the model

In [5]:
# make predictions and evaluate

#training. This section requires a lot of ram so it might be good to comment it out
rmse_value_val, r2_val = predict_stacked_model(stacked_model, features_training, target_training)
print(rmse_value_val, r2_val)
rmse_value_val, r2_val = predict_stacked_model(best_model, features_training, target_training)
print(rmse_value_val, r2_val)

# validation
rmse_value_val, r2_val = predict_stacked_model(stacked_model, feature_validation, target_validation)
print(rmse_value_val, r2_val)
rmse_value_val, r2_val = predict_stacked_model(best_model, feature_validation, target_validation)
print(rmse_value_val, r2_val)

# average of the models instead of concatenating all of them and then training the ensemble model
# # models = [fit_model(features_training, target_training, hidden_layer_size = i, epochs = epochs) for j in range(start_layers, finish_layers + 1) for i in range(number_of_same_model + 1)]
# RMSE_value = evaluate_members(models, feature_validation, target_validation)
# print(RMSE_value)

NameError: name 'load_model' is not defined